# Notes

code for producing GGIT pipelines summary stats, and for calculating landing page stats

this is saved as an Excel file, which Baird copies/pastes into the existing summary tables information on the drive here:
https://docs.google.com/spreadsheets/d/1OYH6D7c-D0FsL5GzBGijtkmvQCTkBUclj-UVoOieUFo/edit

In [326]:
import pandas
import numpy
import pygsheets
import datetime
import re
import pytz

In [350]:
# define the excel file to save tables in
current_time = datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d_T%H%M%S")

In [373]:
#fuel_type = 'Gas'
fuel_type = 'Hydrogen'
#fuel_type = 'Oil'
#fuel_type = 'NGL'

## import data

In [374]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
#spreadsheet = gc.open_by_key('1WaBMIdfRWqSqXUw7_cKXo3RipyhPdnNN8flqEYfMZIA') # file to use for gas pipelines Dec 2023
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek') # CURRENT sheet

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A3')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A3')

gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
oil_pipes = oil_pipes.drop('WKTFormat', axis=1)
pipes_df_orig = gas_pipes.copy() #pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
owners_df_orig = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df(start='A2')

country_ratios_df = country_ratios_df.loc[country_ratios_df.Wiki!='']

# remove empty cells for pipes, owners
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['PipelineName']!='']
#pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['Wiki']!='']
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig.Fuel.isin([fuel_type])]

owners_df_orig = owners_df_orig.loc[owners_df_orig['ProjectID']!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig['PipelineName']!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig.Status!='N/A']

owners_df_orig.set_index('ProjectID', inplace=True)

parent_metadata_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A2')
parent_metadata_df.set_index('Parent', inplace=True)

In [377]:
country_ratios_df.loc[country_ratios_df.Fuel=='Hydrogen'].MergedKmByCountry.sum()

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [353]:
country_ratios_df.replace('--', numpy.nan, inplace=True)

owners_df_orig.replace('',numpy.nan,inplace=True)
owners_df_orig.replace('--',numpy.nan,inplace=True)

pipes_df_orig.replace('--',numpy.nan,inplace=True)

In [354]:
region_df_orig = spreadsheet.worksheet('title', 'Country dictionary').get_as_df(start='A2')

#region_name = 'Global'; region_df_touse = region_df_orig.copy()
#region_name = 'AsiaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AsiaGasTracker=='Yes']
region_name = 'EuroGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.EuroGasTracker=='Yes']
#region_name = 'AfricaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AfricaGasTracker=='Yes']
#region_df_agt.copy()

#region_df_touse = region_df_orig.copy()

In [355]:
region_df_touse_cleaned = region_df_touse.loc[(region_df_touse.Region!='--')&
                                            (region_df_touse.SubRegion!='--')]
multiindex_region_subregion = region_df_touse_cleaned.groupby(['Region','SubRegion'])['Country'].count().index
multiindex_region_subregion

MultiIndex([(  'Asia',    'Western Asia'),
            ('Europe',  'Eastern Europe'),
            ('Europe', 'Northern Europe'),
            ('Europe', 'Southern Europe'),
            ('Europe',  'Western Europe')],
           names=['Region', 'SubRegion'])

## file names with regional specifics

In [356]:
if fuel_type=='Gas':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-'+fuel_type+'-pipelines-'+str(datetime.date.today())+'.xlsx')
if fuel_type=='NGL':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-NGL-pipelines-'+str(datetime.date.today())+'.xlsx')
if fuel_type=='Oil':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-Oil-pipelines-'+str(datetime.date.today())+'.xlsx')
if fuel_type=='Hydrogen':
    excel_writer = pandas.ExcelWriter(region_name+'-summary-sheets-'+fuel_type+'-pipelines-'+str(datetime.date.today())+'.xlsx')

### create country-specific dataframes for region, country_ratios_df, owners_df

In [357]:
country_ratios_df_touse = country_ratios_df.loc[country_ratios_df['Country'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

# owners_df_touse = owners_df_orig.loc[owners_df_orig['Countries'].str.contains(
#                                             '|'.join(region_df_touse['Country'].tolist()))]

pipes_df_touse = pipes_df_orig.loc[pipes_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

# pipes_df_touse = pipes_df_orig.loc[pipes_df_orig.EuroGasTracker=='yes']

In [358]:
pipes_df_touse.Fuel.unique()

array(['Gas'], dtype=object)

### sum MergedKmByCountry and MergedKmByRegion

In [359]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(list(set(country_ratios_df_touse['Region'])))

In [360]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']
excel_status_list_with_countries = ['Country']+excel_status_list

## gas

In [361]:
country_ratios_df_subset = country_ratios_df_touse.loc[country_ratios_df_touse.Fuel.isin([fuel_type])]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=multiindex_region_subregion)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby(['Region','SubRegion'])['MergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['In Development (Proposed + Construction)'] = km_by_region[['Proposed','Construction']].sum(axis=1)
km_by_country['In Development (Proposed + Construction)'] = km_by_country[['Proposed','Construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.names = ['Region','Subregion']
km_by_country.index.name = 'Country'

km_by_region.loc['Total',:] = km_by_region.sum(axis=0).values
km_by_country.loc['Total',:] = km_by_country.sum(axis=0).values

# drop all-zero rows
km_by_country = km_by_country.loc[~(km_by_country==0).all(axis=1)]

km_by_country.replace(0,'',inplace=True)
km_by_region.replace(0,'',inplace=True)

km_by_region.to_excel(excel_writer, 'Kilometers by region')
km_by_country.to_excel(excel_writer, 'Kilometers by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


In [362]:
km_by_region

Proposed  Construction  \
Region Subregion                                 
Asia   Western Asia      1261.86         61.42   
Europe Eastern Europe    3030.07        864.57   
       Northern Europe    972.49          1.20   
       Southern Europe   8237.41        869.00   
       Western Europe     678.30         82.00   
Total                   14180.13       1878.19   

                        In Development (Proposed + Construction)  Shelved  \
Region Subregion                                                            
Asia   Western Asia                                      1323.28   649.28   
Europe Eastern Europe                                    3894.64  3307.27   
       Northern Europe                                    973.69  1000.00   
       Southern Europe                                   9106.41   764.54   
       Western Europe                                     760.30    69.25   
Total                                                   16058.32  5790.34   

                        Cancelled  Operating     Idle  Mothballed  Retired  
Region Subregion                                                            
Asia   Western Asia       4117.99   12528.21               842.00           
Europe Eastern Europe     3571.39   28222.84  1237.38     1599.93   617.42  
       Northern Europe    1003.77   25433.17    155.0     2033.45   564.27  
       Southern Europe    4436.09   27851.40               242.61           
       Western Europe     2138.20   28610.48     8.35      171.81           
Total                    15267.44  122646.10  1400.73     4889.80  1181.69

## pipeline km by parent company (owner) and project status

### first check that there are no missing projectids

In [363]:
owner_parent_calculations_df = pandas.DataFrame()
# needs country, km in each country columns as well

for idx,row in country_ratios_df_subset.iterrows():
    #print(row.ProjectID)
    parent_string = row.Parent
    #print(parent_string)

    #print(parent_string)
    # if the first letter of the parent_string is a chinese character, and it ends with [100.00%],
    # that means it's a non-researched QCC owner; keep as-is
    if re.findall(r'[\u4e00-\u9fff]+', parent_string[:1]) != [] and parent_string[-9:]=='[100.00%]':
        parent_list = [parent_string.split(' [100.00%]')[0]]
        percent_list = [1.0]
    # otherwise do the rest of the thing
    else:
        parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have a Owner [%] syntax
        percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    if parent_list.__len__()!=percent_list.__len__():
        #print(parent_list)
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        else:
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing
    for p_idx,parent in enumerate(parent_list):
        owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                      pandas.DataFrame([{'Parent':parent, 'ProjectID':row.ProjectID, 
                                                                         'FractionOwnership':percent_list[p_idx],
                                                                         'ParentHQCountry':parent_metadata_df.loc[parent,'ParentHQCountry'],
                                                                         'ParentHQRegion':parent_metadata_df.loc[parent,'ParentHQRegion'],
                                                                         'Country':row.Country,
                                                                         'Status':row.Status,
                                                                         'MergedKmByCountry':row.MergedKmByCountry}])])

owner_parent_calculations_df['KmOwnership'] = owner_parent_calculations_df.FractionOwnership*owner_parent_calculations_df.MergedKmByCountry

In [364]:
owner_parent_calculations_df

,Parent,ProjectID,FractionOwnership,ParentHQCountry,ParentHQRegion,Country,Status,MergedKmByCountry,KmOwnership
0,Eni S.p.A.,P0439,0.50,Italy,Europe,Malta,Operating,160.25,80.1250
0,Libyan National Oil Corporation,P0439,0.50,Libya,Africa,Malta,Operating,160.25,80.1250
0,Eni S.p.A.,P0439,0.50,Italy,Europe,Italy,Operating,66.32,33.1600
0,Libyan National Oil Corporation,P0439,0.50,Libya,Africa,Italy,Operating,66.32,33.1600
0,Enagás,P0453,0.25,Spain,Europe,Spain,Mothballed,242.61,60.6525
...,...,...,...,...,...,...,...,...,...
0,unknown,P5985,1.00,unknown,--,Poland,Operating,0.00,0.0000
0,unknown,P5986,1.00,unknown,--,Poland,Operating,0.00,0.0000
0,unknown,P5988,1.00,unknown,--,Poland,Operating,0.00,0.0000
0,unknown,P5988,1.00,unknown,--,Lithuania,Operating,0.00,0.0000


In [365]:
#unique_owner_list = owner_parent_calculations_df.Parent.sort_values().unique().tolist()

##################################################
# create km count by owner, status
##################################################
owners_km_by_status_df = \
    owner_parent_calculations_df.groupby(
    ['Parent','ParentHQCountry','Status'])[['KmOwnership']].sum().unstack().droplevel(axis=1, level=[0])

owners_km_by_status_df = owners_km_by_status_df.reindex(columns=status_list)
owners_km_by_status_df = owners_km_by_status_df.reset_index().set_index('Parent')
owners_km_by_status_df.columns.name = None

owners_km_by_status_df['In Development (Proposed + Construction)'] = owners_km_by_status_df[['Proposed','Construction']].sum(axis=1)

owners_km_by_status_df = owners_km_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_km_by_status_df = owners_km_by_status_df[excel_status_list_with_countries]

# totals_row = owners_ntrains_by_status_df.sum(axis=0, min_count=0)
# totals_row.name = 'Total'
# owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)
owners_km_by_status_df.loc['Total',:] = owners_km_by_status_df.sum(axis=0, min_count=0).values
owners_km_by_status_df.loc['Total','Country'] = ''

owners_km_by_status_df = owners_km_by_status_df.replace(numpy.nan, '')
owners_km_by_status_df = owners_km_by_status_df.replace(0, '')

owners_km_by_status_df.to_excel(excel_writer, sheet_name='Kilometers by owner')

In [366]:
owners_km_by_status_df

,Country,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Parent,,,,,,,,,,
ABRDN Plc,United Kingdom,,,,,,94.0,,,
ADIA,United Arab Emirates,4.25,4.845,9.095,,,435.421225,,,
Acciona S.A,Spain,5.0,,5.0,,,,,,
Agence des participations de l'État,France,,,,,513.14,,,,
Albgaz Sha,Albania,348.5,,348.5,,,,,,
...,...,...,...,...,...,...,...,...,...,...
Zweckverband Oberschwäbische Elektrizitätswerke,Germany,,,,,118.410346,,,,
other,unknown,219.441223,16.8,236.241223,137.689121,160.620346,1228.101186,,,
unknown,unknown,1159.97,261.57,1421.54,134.0,,1330.17,,,


### pipeline km by start year, type

In [367]:
pipes_started = pipes_df_touse.copy()
#pipes_started['StartYearLatest'].replace(numpy.nan,'',inplace=True)

if fuel_type == 'Gas':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='Gas')]
if fuel_type == 'Oil':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='Oil')]
if fuel_type == 'NGL':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['Operating'])) &
                              (pipes_started['Fuel']=='NGL')]

pipes_started_sum = pipes_started.groupby('StartYearEarliest')['LengthMergedKm'].sum()

In [368]:
if fuel_type == 'Gas':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2024)), columns=['Gas pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Gas pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'Oil':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2024)), columns=['Oil pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Oil pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'NGL':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2024)), columns=['NGL pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['NGL pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'Hydrogen':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2024)), columns=['Hydrogen pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Hydrogen pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

km_by_start_year.loc['Total',:] = km_by_start_year.sum(axis=0)

km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')
#km_by_start_year

## save excel file

In [369]:
excel_writer.close()

## calculating stats for landing page

In [370]:
# number of projects tracked in total
print(pipes_df_touse.shape[0], 'gas pipeline projects tracked')
print(pipes_df_touse['LengthMergedKm'].sum()/1e6, 'M km tracked')

899 gas pipeline projects tracked
0.20703458999999996 M km tracked


In [371]:
# number of projects tracked in total
print(pipes_df_touse.loc[pipes_df_touse.Fuel==fuel_type].shape[0], 'gas pipeline projects tracked')
print(pipes_df_touse.loc[pipes_df_touse.Fuel==fuel_type]['LengthMergedKm'].sum()/1e6, 'M km tracked')

899 gas pipeline projects tracked
0.20703458999999996 M km tracked


In [372]:
# number of projects tracked in total
print(pipes_df_touse.loc[(pipes_df_touse.Fuel==fuel_type)&
                        (pipes_df_touse.Status.isin(['Proposed','Construction']))].shape[0], 'gas pipeline projects tracked')
print(pipes_df_touse.loc[(pipes_df_touse.Fuel==fuel_type)&
                        (pipes_df_touse.Status.isin(['Proposed','Construction']))]['LengthMergedKm'].sum()/1e3, 'K km tracked')

152 gas pipeline projects tracked
23.682470000000002 K km tracked
